In [1]:
import gc
import json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm_notebook
from glob import glob

from utils import loadpkl, to_json, scalingPredictions, getBestMultiple

%matplotlib inline

/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [29]:
# search a best weight for 2 predictions
def getBestWeights(act, pred_lgbm, pred_xgb):
    search_range = np.arange(0.0, 1.1, 0.1)
    best_weights = []

    # base prediction
    _pred = pd.DataFrame()
    cols_pred=[]
    for i in range(0,12):
        _pred['pred_{}'.format(i)] = 0.5*pred_lgbm['pred_lgbm_plans{}'.format(i)]+ 0.5*pred_xgb['pred_xgb_plans{}'.format(i)]
        cols_pred.append('pred_{}'.format(i))

    # base score
    best_f1=f1_score(act, np.argmax(_pred[cols_pred].values,axis=1),average='weighted')
    
    # get best weights for each classes
    for i in range(0,12):
        f1s = []
        for _w in search_range:
            tmp_pred = _pred[cols_pred]
            tmp_pred['pred_{}'.format(i)] = _w*pred_lgbm['pred_lgbm_plans{}'.format(i)]+ (1.0-_w)*pred_xgb['pred_xgb_plans{}'.format(i)]
                    
            # calc f1 score
            _f1 = f1_score(act, np.argmax(tmp_pred.values,axis=1),average='weighted')
            f1s.append(_f1)
            print('class: {}, w: {}, f1 score: {}'.format(i,_w,_f1))
            if _f1 > best_f1:
                best_f1 = _f1
                best_w = _w

            del tmp_pred

        # save weights & predicted values
        best_weights.append(best_w)
        _pred['pred_{}'.format(i)] = best_w*pred_lgbm['pred_lgbm_plans{}'.format(i)]+ (1.0-best_w)*pred_xgb['pred_xgb_plans{}'.format(i)]
        
        # plot thresholds
        plt.figure()
        plt.plot(search_range, f1s)
        plt.savefig('../imp/multiple{}.png'.format(i))

    return best_weights

In [4]:
sub1 = pd.read_csv('../output/submission_lgbm_1.csv')
sub2 = pd.read_csv('../output/submission_lgbm_2.csv')
sub3 = pd.read_csv('../output/submission_lgbm_3.csv')

In [6]:
sub = sub1.append(sub2)
sub = sub.append(sub3)

In [7]:
sub.to_csv('../output/submission_split.csv',index=False)